In [63]:
import numpy as np
import matplotlib.pyplot as plt 
import scipy as sc
from scipy import optimize
from mpl_toolkits.mplot3d import Axes3D
import random as rnd
from scipy.optimize import minimize
import scipy.linalg
from scipy.optimize import Bounds
import multiprocessing

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

#calculate Ginzburg parameter
Ginzburg = True
#calculating Ising wall
Ising = False

prefix = "test"
data_write = True
data_read = False

#numerical minimization parameters
#number of points in grid
nsteps = 100
#length between grid points
dx = 0.8
#amplitude of initial uniform q
q_init = 1
#scf convergence threshold
scf_conv = 1e-4
#scf mixing factor
scf_mix = 0.7

parallel = False

#volume of unit cell in angstrom cubed for volume normalization
volume = 7.3236460686 * 7.7495899200 * 7.3236460686


#_________________________________PARAMETERS__________________________________________#

#Landau parameters for 4th order fit
a_Q_4th = -0.161
b_Q_4th = 0.2081

#Ginzburg parameter
#s

# modenumbers corresponding to GM2+ starting at 1
# 5, 7, 14, 16, 23, 31, 49, 69, 71, 74, 77, 83
modenumber = 1

#_________________________________________GINZBURG___________________________________________#
#____________________________________________________________________________________________#
ry_to_mev = 13.605698066 * 1000
ry_to_thz = 3289.845
thz_to_mev = 0.00414E3
if Ginzburg:

    #before calculating the Ginzburg parameter we make sure that everything is correct with the calculated dynamical
    #matrices and the calculated eigenvectors by reproducing the frequencies by diagonalizing the calculated dynamical matrices
    #and by obtaining then from the calculated eigenvectors and the calculated dynamical matrices
    
    #___________READING DYNMATS_____________#
    #Note that we are actually reading the force constant matrices here not the dynamical matrices!
    #dynmat at q_GM
    dynmat_0_temp = np.loadtxt("DATA/DYNMAT/dynmat_0")
    #dynmat at q_GM - dq
    dynmat_pqa_temp = np.loadtxt("DATA/DYNMAT/dynmat_mqa")
    #dynmat at q_GM - dq
    dynmat_mqa_temp = np.loadtxt("DATA/DYNMAT/dynmat_mqa")
    
    num_of_atoms = dynmat_0_temp.shape[0]
    num_of_atoms = int(np.sqrt(num_of_atoms/3))
    
    #dynamical matrix:
    # -index 1: floor(index/3) -> atom index 1
    #         index%3 -> coordinate index of corresponding atom
    # -index 2: floor(index/3) -> atom index 2
    #         index%3 -> coordinate index of corresponding atom
    
    dynmat_0 = np.empty([num_of_atoms*3,num_of_atoms*3], dtype = complex)
    dynmat_pqa = np.empty([num_of_atoms*3, num_of_atoms*3], dtype = complex)
    dynmat_mqa = np.empty([num_of_atoms*3, num_of_atoms*3], dtype = complex)
    temp_index1 = 0
    for i in range(0, num_of_atoms):          #block index atom 1
        for j in range(0, 3):                 #coordinate index
            temp_index2 = 0
            for k in range(0, num_of_atoms):  #block index atom 2 in atom 1
                shift = i*num_of_atoms*3 + j + k*3
                for l in range(0,3):
                    dynmat_0[temp_index1, temp_index2] = np.complex(dynmat_0_temp[shift, l*2], dynmat_0_temp[shift, l*2 + 1])
                    dynmat_pqa[temp_index1, temp_index2] = np.complex(dynmat_pqa_temp[shift, l*2], dynmat_pqa_temp[shift, l*2 + 1])
                    dynmat_mqa[temp_index1, temp_index2] = np.complex(dynmat_mqa_temp[shift, l*2], dynmat_mqa_temp[shift, l*2 + 1])
                    temp_index2 += 1
            temp_index1 += 1
          
                
    #_____________DYNAMICAL MATRIX MASS NORMALIZATION______________#
    #actual conversion from the force constant matrices to the dynamical matrices
    mass1 = 0
    mass2 = 0
    for i in range(0, num_of_atoms * 3):
        if i < 8*3:
            mass1 = 167559.909473597
        else:
            mass1 = 14582.5610075711
        for j in range(0, num_of_atoms*3):
            if j < 8*3:
                mass2 = 167559.909473597
            else:
                mass2 = 14582.5610075711
            
            dynmat_0[i,j] /= np.sqrt(mass1 * mass2)
            dynmat_pqa[i,j] /= np.sqrt(mass1 * mass2)
            dynmat_mqa[i,j] /= np.sqrt(mass1 * mass2)
            
    #____________IMPOSE DYNMAT HERMITICITY_____________#
    for i in range(0, num_of_atoms * 3):
         for j in range(0, i):
                dynmat_0[i,j] = 0.5 * (dynmat_0[i,j] + dynmat_0[j,i].conjugate())
                dynmat_0[j,i] = dynmat_0[i,j].conjugate()
                dynmat_pqa[i,j] = 0.5 * (dynmat_pqa[i,j] + dynmat_pqa[j,i].conjugate())
                dynmat_pqa[j,i] = dynmat_pqa[i,j].conjugate()
                dynmat_mqa[i,j] = 0.5 * (dynmat_mqa[i,j] + dynmat_mqa[j,i].conjugate())
                dynmat_mqa[j,i] = dynmat_mqa[i,j].conjugate()
    
    #_________DIAGONALIZE DYNMATS WITH ZHEEV____________#
    w2h_0, evh_0, inf = scipy.linalg.lapack.zheev(dynmat_0)
    w2h_pqa, evh_pqa, inf = scipy.linalg.lapack.zheev(dynmat_pqa)
    w2h_mqa, evh_mqa, inf = scipy.linalg.lapack.zheev(dynmat_mqa)
    w2h_0 *= ry_to_thz**2
    w2h_pqa *= ry_to_thz**2
    w2h_mqa *= ry_to_thz**2    
    
    #_________CALCULATE FREQUENCIES______________#    
    w2h_0_temp = w2h_0
    w2h_pqa_temp = w2h_pqa
    w2h_mqa_temp = w2h_mqa    
    wh_0 = np.sqrt(abs(w2h_0))
    wh_pqa = np.sqrt(abs(w2h_pqa))
    wh_mqa = np.sqrt(abs(w2h_mqa))
    for i in range(0, wh_0.size):
        if w2h_0_temp[i] < 0:
            wh_0[i] = -wh_0[i]
        if w2h_pqa_temp[i] < 0:
            wh_pqa[i] = -wh_pqa[i]
        if w2h_mqa_temp[i] < 0:
            wh_mqa[i] = -wh_mqa[i]
                    
    #____________CALCULATE FREQUENCIES FROM READ EIGENVECTORS_________________#    
    #____________READING EIGENVECTORS_____________#
    evr_0_temp = np.loadtxt("DATA/DYNMAT/eigenvector_0")
    evr_pqa_temp = np.loadtxt("DATA/DYNMAT/eigenvector_pqa")
    evr_mqa_temp = np.loadtxt("DATA/DYNMAT/eigenvector_mqa")
    
    #eigenvectors
    # -index: floor(index/3) -> atom index
    #         index%3 -> coordinate index of corresponding atom
    evr_0 = np.empty(shape=(num_of_atoms * 3, num_of_atoms * 3), dtype = complex)
    evr_pqa = np.empty(shape=(num_of_atoms * 3, num_of_atoms * 3), dtype = complex)
    evr_mqa = np.empty(shape=(num_of_atoms * 3, num_of_atoms * 3), dtype = complex)

    for i in range(0, num_of_atoms * 3):
        temp_index = 0
        for j in range(0, num_of_atoms):
            for k in range(0, 3):
                evr_0[i, temp_index] = np.complex(evr_0_temp[i*num_of_atoms+j, k*2], evr_0_temp[i*num_of_atoms+j, k*2 + 1])
                evr_pqa[i, temp_index] = np.complex(evr_pqa_temp[i*num_of_atoms+j, k*2], evr_pqa_temp[i*num_of_atoms+j, k*2 + 1])
                evr_mqa[i, temp_index] = np.complex(evr_mqa_temp[i*num_of_atoms+j, k*2], evr_mqa_temp[i*num_of_atoms+j, k*2 + 1])
                temp_index += 1
                
    w2r_0 = np.empty(num_of_atoms * 3)
    w2r_pqa = np.empty(num_of_atoms * 3)
    w2r_mqa = np.empty(num_of_atoms * 3)
    
    wr_0 = np.empty(num_of_atoms * 3)
    wr_pqa = np.empty(num_of_atoms * 3)
    wr_mqa = np.empty(num_of_atoms * 3)
    

    for i in range(0, num_of_atoms * 3):
        #remultiply the read eigenvectors by the sqrt of the mass as the eigenvectors are actually the eigenvectors
        #of the force constant matrix
        for j in range(0, num_of_atoms * 3):
            if j < 8*3:
                mass = 167559.909473597
            else:
                mass = 14582.5610075711
            evr_0[i,j] *= np.sqrt(mass)
            evr_pqa[i,j] *= np.sqrt(mass)
            evr_mqa[i,j] *= np.sqrt(mass)   
         
        #normalize their lengths
        evr_0[i] /= np.sqrt(evr_0[i].dot(evr_0[i].conjugate()))
        evr_pqa[i] /= np.sqrt(evr_pqa[i].dot(evr_pqa[i].conjugate()))
        evr_mqa[i] /= np.sqrt(evr_mqa[i].dot(evr_mqa[i].conjugate()))
            
        #calculate squared frequencies
        w2r_0[i] = evr_0[i].dot(dynmat_0.dot(evr_0[i]))
        w2r_pqa[i] = evr_pqa[i].dot(dynmat_pqa.dot(evr_pqa[i]))
        w2r_mqa[i] = evr_mqa[i].dot(dynmat_mqa.dot(evr_mqa[i]))
        
        #convert squared frequencies from Ry to THz
        w2r_0[i] *= ry_to_thz**2  
        w2r_pqa[i] *= ry_to_thz**2 
        w2r_mqa[i] *= ry_to_thz**2
        
        #get the frequencies from the squared frequencies
        wr_0_temp = np.sqrt(abs(w2r_0[i]))
        wr_pqa_temp = np.sqrt(abs(w2r_pqa[i]))
        wr_mqa_temp = np.sqrt(abs(w2r_mqa[i]))
        if w2r_0[i] < 0:
            wr_0_temp = -wr_0_temp
        if w2r_pqa[i] < 0:
            wr_pqa_temp = -wr_pqa_temp
        if w2r_mqa[i] < 0:
            wr_mqa_temp = -wr_mqa_temp
        
        wr_0[i] = wr_0_temp
        wr_pqa[i] = wr_pqa_temp
        wr_mqa[i] = wr_mqa_temp

    
    #compare frequencies from read and calculated eigenvectors to make sure everything is correct
    print("Mode frequencies in THz:")
    print(" q0: %f (%f)" % (wr_0[modenumber], wh_0[modenumber]))
    print(" q-: %f (%f)" % (wr_pqa[modenumber], wh_pqa[modenumber]))
    print(" q+: %f (%f)" % (wr_mqa[modenumber], wh_mqa[modenumber]))
    
    
    
    #__________COMPUTE THE GINZBURG PARAMETER______________#
    #we need the force constant matrix and its eigenvectors so ...

    #... we convert the dynamical matrices back to the force constant matrices by multiplying with the masses
    for i in range(0, num_of_atoms * 3):
        if i < 8*3:
            mass1 = 167559.909473597
        else:
            mass1 = 14582.5610075711
        for j in range(0, num_of_atoms*3):
            if j < 8*3:
                mass2 = 167559.909473597
            else:
                mass2 = 14582.5610075711
            
            dynmat_0[i,j] *= np.sqrt(mass1 * mass2)
            dynmat_pqa[i,j] *= np.sqrt(mass1 * mass2)
            dynmat_mqa[i,j] *= np.sqrt(mass1 * mass2)
            
    #____________GET FORCE CONSTANT MATRIX EIGENVALUES AND EIGENVECTORS___________#
    w2f_0, evf_0, inf = scipy.linalg.lapack.zheev(dynmat_0)
    w2f_mqa, evf_mqa, inf = scipy.linalg.lapack.zheev(dynmat_mqa)
    w2f_pqa, evf_pqa, inf = scipy.linalg.lapack.zheev(dynmat_pqa) 

    #normalize length of eigenvectors
    for iev in evf_0:
        iev /= np.sqrt(iev.dot(iev.conjugate()))
    for iev in evf_mqa:
        iev /= np.sqrt(iev.dot(iev.conjugate()))
    for iev in evf_pqa:
        iev /= np.sqrt(iev.dot(iev.conjugate()))
        

#============================================================================================#

    evr_0_temp = np.loadtxt("DATA/DYNMAT/eigenvector_0")
    evr_pqa_temp = np.loadtxt("DATA/DYNMAT/eigenvector_pqa")
    evr_mqa_temp = np.loadtxt("DATA/DYNMAT/eigenvector_mqa")
    
    #eigenvectors
    # -index: floor(index/3) -> atom index
    #         index%3 -> coordinate index of corresponding atom
    evr_0 = np.empty(shape=(num_of_atoms * 3, num_of_atoms * 3), dtype = complex)
    evr_pqa = np.empty(shape=(num_of_atoms * 3, num_of_atoms * 3), dtype = complex)
    evr_mqa = np.empty(shape=(num_of_atoms * 3, num_of_atoms * 3), dtype = complex)

    for i in range(0, num_of_atoms * 3):
        temp_index = 0
        for j in range(0, num_of_atoms):
            for k in range(0, 3):
                evr_0[i, temp_index] = np.complex(evr_0_temp[i*num_of_atoms+j, k*2], evr_0_temp[i*num_of_atoms+j, k*2 + 1])
                evr_pqa[i, temp_index] = np.complex(evr_pqa_temp[i*num_of_atoms+j, k*2], evr_pqa_temp[i*num_of_atoms+j, k*2 + 1])
                evr_mqa[i, temp_index] = np.complex(evr_mqa_temp[i*num_of_atoms+j, k*2], evr_mqa_temp[i*num_of_atoms+j, k*2 + 1])
                temp_index += 1
        
    for iev in evr_0:
        iev /= np.sqrt(iev.dot(iev.conjugate()))
    for iev in evr_mqa:
        iev /= np.sqrt(iev.dot(iev.conjugate()))
    for iev in evr_pqa:
        iev /= np.sqrt(iev.dot(iev.conjugate()))
        
#============================================================================================#
    
    #compute the second derivative of the force constant matrix centered at q0
    dd_dynmat = dynmat_pqa + dynmat_mqa - 2*dynmat_0
    dd_dynmat /= 0.01 ** 2
    
    #compute s from the second derivative of the force constant matrix 
    sf = evf_0[modenumber].conjugate().dot(dd_dynmat.dot(evf_0[modenumber]))
    sr = evr_0[modenumber].conjugate().dot(dd_dynmat.dot(evr_0[modenumber]))

    print(sf, sr)
    #conversion from ry to meV, taking care of Taylor factor 1/2 and volume normalization
    sf *= ry_to_mev / (2*volume)    
    sf = float(sf)    
    sr *= ry_to_mev / (2*volume)    
    sr = float(sr)    

   
    #compute s from the eigenvalues of the force constant matrix
    sw = w2f_mqa[modenumber] + w2f_pqa[modenumber] - 2*w2f_0[modenumber]
    sw /= 0.01 ** 2
    sw *= ry_to_mev / volume
    
    print("Ginzburg parameter in meV:")
    print(" s: %f %f (%f)" % (sf, sr, sw))

#____________________________________________________________________________________________#
#____________________________________________________________________________________________#

#______________________________________DOMAIN_WALLS__________________________________________#
#____________________________________________________________________________________________#


#_______________________________________________________________________#

#__________________________ISING/NO CHARGE______________________________#

if Ising and data_write:
    
    def compute_Ising(parallel=False):
        #____________NUMERICAL______________#
        Q = np.zeros(nsteps) - q_init
        Q[int(Q.size/2) : Q.size] = q_init
        
        #x grid of size nsteps*dx centered at 0
        x = np.linspace(0, nsteps*dx, nsteps)
        x -= x[x.size - 1] / 2 
        
        def Landau_Ginzburg(Q):
            d_Q = np.diff(Q)/dx
            d_Q = np.append(d_Q, d_Q[0])
            E_tot = 0
            for i in range(0, Q.size - 1):
                Landau = a_Q_4th*(Q[i]**2) + b_Q_4th*(Q[i]**4)
                Ginzburg = s * (d_Q[i]**2)
                E_tot += (Landau + Ginzburg) * dx
            return E_tot
        
        Q_min = minimize(Landau_Ginzburg, Q, method='BFGS')
        Q_min = Q_min.x
            
        #____________ANALYTICAL______________#
        epsilon = np.sqrt(-s / a_Q_4th )
        Q0 = np.sqrt(- a_Q_4th / (2* b_Q_4th))    
        analytical = np.array(Q0*np.tanh(x/(np.sqrt(2)*epsilon)))
    
        if not parallel:
            #_______________PLOT_________________#    
            plt.figure(10, figsize=[10,10])
            plt.plot(x, Q_min, color='blue', linewidth=5, label='numerical')
            plt.plot(x, analytical, color='red', linestyle='-.', linewidth=5, label='analytical')
        
            plt.xlabel(r'$x [\mathring{A}]$', fontsize=30)
            plt.ylabel(r'$q [\mathring{A}]$', fontsize=30)
        
            plt.tight_layout()
            plt.savefig('Landau_Ginzburg.eps', bbox_inches='tight')   
        
        #___________WRITE TO FILE____________#
        file_num = open('DATA/DOMAIN_WALLS/' + prefix + '.Ising_num', 'w')
        file_ana = open('DATA/DOMAIN_WALLS/' + prefix + '.Ising_ana', 'w')
        for i in range(0, len(x)):
            file_num.write("%f\t%f\n" %(x[i], Q_min[i]))
            file_ana.write("%f\t%f\n" %(x[i], analytical[i]))
        file_num.close()
        file_ana.close() 
        
    if parallel:
        proc_Ising = multiprocessing.Process(target=compute_Ising, args=(parallel,))
        proc_Ising.start()
    else:
        compute_Ising()
    
    
#_________________________________________________________________________#

        
if data_write and parallel:
    if Ising:
        proc_Ising.join()
        
#_________________________________________________________________________________#
#___________________________PLOTTING FROM FILE____________________________________# 

if Ising and data_read:
    x_Q_num = np.loadtxt('DATA/DOMAIN_WALLS/' + prefix + '.Ising_num')
    x = x_Q_num[:,0]
    Q_num = x_Q_num[:,1]
    
    x_Q_ana = np.loadtxt('DATA/DOMAIN_WALLS/' + prefix + '.Ising_ana')
    Q_ana = x_Q_ana[:,1]
    
    plt.figure(17, figsize=[10,10])
    
    plt.plot(x, Q_num, color='blue', label='numerical', linewidth=5)
    plt.plot(x, Q_ana, color='red', label='analytical', linewidth=5, linestyle='-.')
  
    plt.xlabel('r$x [\mathring{A}]$', fontsize=30)
    plt.ylabel('r$Q [\mathring{A}]$', fontsize=30)
    plt.legend()
    plt.tight_layout()
    
plt.show()
print("done")

Mode frequencies in THz:
 q0: 0.365945 (0.365945)
 q-: -1.072112 (-1.079008)
 q+: -1.072112 (-1.079008)
(1091.82260778947-3.774758283725532e-15j) (-170.43466695220653-1.7763568394002505e-15j)
Ginzburg parameter in meV:
 s: 17869.380579 -2789.429259 (-1496.013154)
done
